# KDE Custom Motor Head

**Goal:** The KDE motor overheats while running. If we increase airflow over the coils, can we improve the runtime of the motor add different power settings?

For our 3D printed part, running it with a prop would tear it apart. Therefore, initial tests should be run with a prop. The main goal of a test should be to show that the rates of temperature change are reduced with the custom head.

The KDE website states that their components are designed to have a 5 mph windowflow when they are run. However, we can not measure the effect the fan's airflow provides. As a result, we should use a strict upper and lower bound on temperature.

## Test Structure

The test will consist of decreasing throttle steps with cooling in between. 
Initially, the motor will be heated to the starting temperature.
Then the motor will be run at the max throttle until the end of a time period, or the thermal safety cutoff is hit. 
The motor will sit to cool back down to the starting temperature before the next throttle setting lower than max is run.

```
Throttle:
 .----.
 |    |   .----.   
 |    |   |    |   .----.
 |    |   |    |   |    |   .----.
_|    |___|    |___|    |___|    |___

```

## Test Parameters
| Param | Value | Reason |
|---|--|---|
| **Start temp**          | 95 \*F      |
| **Safety Cutoff temp**  | 115 \*F     |
| **Max PWM**             | 2000        | 100%
| **Min PWM**             | 1200        | 20%
| **Heating Time**        | 5 mins      | pretty good at separating runaway in KDE-Levels
| **PWM Step**            | -100        | 10% throttle


In [89]:
from analysis_tools import *
import bokeh
from bokeh.plotting import *
output_notebook()

Loading BokehJS ...

In [90]:
unFilteredRuns = runsFromFolder('custom_head_testing')
unFilteredRuns = list(filter(lambda x: 'Fail' not in x.file, unFilteredRuns))

In [91]:
runs = unFilteredRuns
#runs = list(filter(lambda x: 'Mega' in x.file, unFilteredRuns))

In [92]:
for r in runs:
    r.props["cutoffs"] = findSpikes(r, eqThrottle=[2000, 1900, 1800, 1700, 1600, 1500, 1400, 1300, 1200], minThrottle=1000)[1:]

In [93]:
for run in runs:
    testSegments = dict()
    run.props["segments"] = testSegments
    for cutoff in run.props["cutoffs"]:
        testPwm = run.data[cutoff].pwm
        i = cutoff
        while i > 0 and run.data[i].pwm == testPwm:
            i -= 1
        testSegments[int(testPwm)] = (i, cutoff+1)

In [94]:
#plot = figure()

for run in runs:
    pwms = list(run.props["segments"].keys())
    
    plot = figure()

    
    for pwm in pwms:
        color = mapPwmToColor(pwm)
        segment = run.props["segments"][pwm]
        pts = run.data[segment[0]:segment[1]]
    
        startTime = pts[0].time
    
        times = [pt.time - startTime for pt in pts]
        temps = [pt.motorTemp for pt in pts]
    
        plot.line(times, temps, color=color)
        
    show(plot)    
        
#show(plot)    

In [95]:

for run in runs:
    plot = figure()

    
    # plot.line([x for x in range(0, len(run.data))], [x.motorTemp for x in run.data])
    # plot.line([x.time for x in run.data], [(x.pwm - 1000) * (30 / 1000) + 90 for x in run.data], color='gray')
    
    show(plot)

In [96]:
for run in runs:
    run.props['cutons'] = list()

    for i in run.props['cutoffs']:
        while i < len(run.data) and run.data[i].motorTemp > 95:
            i += 1

        if i < len(run.data) and run.data[i].motorTemp <= 95:
            run.props['cutons'].append(i)

In [97]:
#plot = figure()
for run in runs:
    plot = figure(title=run.file)

    for i in range(0, len(run.props['cutons'])):
        
        pwm = run.data[run.props['cutoffs'][i]].pwm
        color = mapPwmToColor(pwm)
        
        tempMin = run.data[run.props['cutons'][i]-1].motorTemp
        coolDuration = run.data[run.props['cutons'][i]-1].time
        
        data = run.data[run.props['cutoffs'][i]:run.props['cutons'][i]]
        plot.line([x.time - coolDuration for x in data], [x.motorTemp - tempMin for x in data], color=color)
        plot.line([x.time - coolDuration for x in data], [x.ambTemp - 80 for x in data], color=color)


    show(plot)
#show(plot)

In [100]:
kdeRun = runs[0]

sections = range(120, 90, -5)
foci = list(zip(sections[:-1], sections[1:]))

plots = dict(map(lambda x: (x, figure(title=str(x))), foci))

for i in kdeRun.props["cutoffs"]:
    pwm = kdeRun.data[i-1].pwm
    
    maxTemp = kdeRun.data[i].motorTemp
    maxTempIndex = i
    while kdeRun.data[i].motorTemp > 95:
        if kdeRun.data[i].motorTemp > maxTemp:
            maxTemp = kdeRun.data[i].motorTemp
            maxTempIndex = i
        i += 1
    
    for focus in foci:
        i = maxTempIndex
    
        while kdeRun.data[i].motorTemp > focus[0]:
            i += 1
            #print(str(kdeRun.data[i].motorTemp) + '\t>\t\t\t' + str(focus[0]))
        segStart = i-1
    
        while kdeRun.data[i].motorTemp > focus[1]:
            i += 1
            #print(str(kdeRun.data[i].motorTemp) + '\t>\t\t\t' + str(focus[1]))
        segEnd = i
        
        segData = kdeRun.data[segStart:segEnd]
        minTemp = segData[-1:][0].motorTemp
        plot = plots[focus]
        plot.line([x.time - segData[-1:][0].time for x in segData], [x.motorTemp - minTemp for x in segData], color=mapPwmToColor(pwm))
    
    


In [101]:
for focus in foci:
    show(plots[focus])